# Práctico 0-A: Manipulación de Grafos en IGraph y Networkx

# Introducción

En este práctico vamos a aprender a manejar grafos usando dos de las librerias más populares: `igraph` y `networkx`. 

Primero vamos a trabajar con `igraph` y luego vamos a repetir mucho de lo ya hecho usando `networkx`. `Igraph` tiene una contraparte en `R` muy importante y es un poco más rápida que `networkx`, pero `networkx` es más popular entre los usuarios de `Python`.


# 1) IGraph

Esta sección se inspira en el Capítulo 2 del libro:
* [SANDR]: Kolaczyk, E.D. and Csárdi, G. "Statistical Analysis of Network Data with R". Use R!, Springer New York, 2014. ISBN 9781493909834

Si aun no tiene el libro, descarguelo, disponible en http://timbo.org.uy/.
Ir a la descarga directamente siguiendo el link (Noviembre, 2019): https://link-springer-com.proxy.timbo.org.uy/book/10.1007%2F978-1-4939-0983-4

Utilizar el manual de referencia del paquete igraph para extender el conocimiento: https://igraph.org/python/doc/api/igraph.html

Comenzamos por instalar igraph (no viene por defecto en Google Colab).

In [1]:
!pip install python-igraph cairocffi

Una vez instalada podemos proceder a importar las libererias que vamos a usar

In [2]:
import igraph as ig

import scipy.io as sio
import numpy as np

## 1.a) Operaciones básicas

Vamos a comenzar con operaciones simples como crear un grafo a partir de una lista de aristas.

`IGraph` requiere que los nodos estén númerados con enteros empezando en 0.

In [3]:
edgelist = [(0,1), (0, 2), (1,2), (2,3), (1,4), (3, 4), (3,5), (4,5), (3,6), (5,6)]
g = ig.Graph()
g.add_vertices(7)
g.add_edges(edgelist)

## or

g = ig.Graph(edgelist)
g.vs["label"] = range(7)

Podemos imprimir información sobre el grafo

In [4]:
print(g)

Así como calcular la cantidad de vértices o aristas

In [5]:
g.vcount(), g.ecount()

Podemos acceder a la lista de todos los vértices usando `g.vs`.
Un vértice es un objeto de `Python`.

In [6]:
[x for x in g.vs]

Y a la  lista de todas las aristas usando `g.es`.

In [7]:
[x for x in g.es]

In [8]:
#listar vertices incidentes a cada arista
[x.tuple for x in g.es]

Podemos definir grafos como dirigidos al crearlos y asociar etiquetas (`label`) a cada nodo.

Los atributos de nodos y aristas pueden accederse como un diccionario.

In [9]:
g2 = ig.Graph(edges=[(0, 1), (0, 2), (1,2), (2, 1)], directed=True)
g2.vs["label"] = range(3)

Podemos gráficar grafos con la función `ig.plot`.

In [10]:
ig.plot(g2, bbox=(0, 0, 100, 100))

Como habiamos dicho, el atributo `label` define las etiquetas a mostrar en el grafo cuando se grafica.

In [11]:
new_labels = ["apple", "carrot", "banana"]
g2.vs["label"] = new_labels
ig.plot(g2, bbox=(0, 0, 200, 200))

Se accede a la ayuda de cada método con la función `help()`.

In [12]:
help(g2.incident)

`IGraph` brinda muchas funciones para calcular propiedades de los nodos y las aristas como son los casos a continuación.

In [13]:
#para cada vértice, imprimo los aristas incidentes (entrantes, salientes y todas), y los vertices de esas aristas

print(g2)

for node in range(3):
  for mode in ["out", "in", "all"]:
    incidents = g2.incident(node, mode=mode)
    print(f"Node {node}, Mode {mode}: ", incidents)
    print([g2.es[e].tuple for e in incidents])

In [14]:
#obtener la lista de adyacencia de un grafo

g2.get_adjacency(), type(g2.get_adjacency())

Podemos construir un subgrafo simplemente listando los nodos que deseamos preservar.

In [15]:
ig.plot(g, bbox=(0, 0, 200, 200)) # Gráfo original

In [16]:
h = g.subgraph([0, 1, 2, 3, 4]) # Subgrafo, a partir de un subconjunto de vértices
ig.plot(h, bbox=(0, 0, 200, 200))

También podemos aplicar transformaciones arbitrarias como agregar o eliminar nodos y aristas

In [17]:
g.delete_vertices(4)
g.delete_edges([(1, 2)])
g.add_vertex("apple", label="apple")
g.add_edge("apple", 1)

In [18]:
ig.plot(g, bbox=(0, 0, 200, 200))

In [19]:
#propiedades del grafo
print('es conexo: ' + str(g.is_connected()))
print('es simple: ' + str(g.is_simple()))

In [20]:
#manejo atributos

g.vs["name"] = g.vs["label"]
g.vs["color"] = ['red','red','red','red','red','red','green']
display([v.attributes() for v in g.vs])

ig.plot(g, bbox=(0, 0, 200, 200))

In [21]:
print('el diámetro es: ' + str(g.diameter()))

Para más detalles en el funcionamiento de la biblioteca pueden acceder a la documentación en el siguiente link: [Python-igraph](https://igraph.org/python/doc/tutorial/)

## 1.b) Explorar una red de comunicaciones: emails de la empresa Enron

La red de emails de la empresa Enron se publicó durante una investigación federal de EE.UU. (si usted esta interesado en leer más sobre el escándalo de Enron, ver http://en.wikipedia.org/wiki/Enron_scandal)

En éste ejercicio haremos cálculos sencillos sobre el grafo de esta red usando `IGraph`.

La red completa está disponible online en http://www.cs.cmu.edu/~enron/ y contiene 517.431 correos
extraidos de los directorios de correo de 150 usuarios. Aqui utilizaremos un grafo más pequeño, preparado
por C. E. Priebe et al., que consiste en 34.427 correos enviados entre 184 correos de empleados de Enron
(ver http://cis.jhu.edu/~parky/Enron/enron.html por más detalles). Los correos son del periodo 13/11/1998
al 21/06/2002 (44 meses).

A continuación incluimos código (usando herramientas de la línea de comando) para descargar el dataset.

In [22]:
!wget "https://github.com/prbocca/na101_master/blob/master/homework_00_a_graphs/Enron.zip?raw=true" -O "Enron.zip"

!unzip -o Enron.zip

In [23]:
Y = sio.loadmat("Y.mat")["Y"]
employees = [x[0][0] for x in sio.loadmat("employees.mat")["employees"]]

display(Y.shape)
display(employees[0:3])

### 1.b.1) Análisis del Dataset

Vamos a comenzar el análisis y para esto tendrá que completar algunos fragmentos de código usted mismo.

Primero, defina dos matrices: $\mathbf{A}$ y $\mathbf{A}_{bin}$. 

$\mathbf{A}\in \mathbb{N}^{184\times184}$ es la matriz de adyacencia con pesos correspondientes a la cantidad de emails en común.

Mientras que $\mathbf{A_{bin}}\in\{0,1\}^{184\times184}$ es la matriz binaria de adyacencia.

In [24]:
### START CODE HERE
### END CODE HERE

display(A.shape)
display(A_bin.shape)

Crearemos el grago `g` a partir de la matriz de adyacencia ponderada, $\mathbf{A}$.

Agregaremos los nombres de los empleados como atributos de los vértices (`label` y `name`) 

In [25]:
g = ig.Graph.Weighted_Adjacency(A.tolist())
g.vs["name"] = employees
g.vs["label"] = employees

ig.summary(g)

Podemos verificar que la cantidad de nodos son 184 y que la cantidad de aristas son 3007.

In [26]:
print(g.vcount(), g.ecount())

In [27]:
#los atributos son muy largos, preferimos usar summary()
print(g)

Más adelante veremos como graficar este grafo. Por ahora, la siguiente es una visualización sencilla.

In [28]:
layout_g=g.layout(layout='auto')

ig.plot(g, layout=layout_g, bbox=(0, 0, 200, 200),
        edge_color="gray", edge_width=1, edge_lty=1, edge_arrow_size=.5, 
        vertex_size=3, vertex_label=None)

Ahora nos familiarizarnos con operaciones básicas sobre grafos.

Para cada uno de las siguientes partes, realizar los cálculos usando la matriz de adyacencia $A$ (con álgebra de matrices), y usando el grafo (con métodos de `IGraph`).



### 1.b.2) Cantidad de aristas (arcos)

Calcular la cantidad de aristas (arcos) en la red.
Es decir, la cantidad de parejas únicas ordenadas $(u,v)\in E$ donde $u,v\in V$.

In [29]:
###
#con matrices
n_edges_A = np.sum(A_bin)

#con igraph
n_edges_lib = g.ecount()
###

print(n_edges_A, n_edges_lib)

### 1.b.3) Cantidad de aristas no dirigidas

Calcular la cantidad de aristas no dirigidas en la red (es decir, la cantidad de parejas únicas no ordenadas $(u,v)\in E$
donde $u,v\in V$. Esto significa que si $(u,v)\in E$ o $(v,u)\in E$, entonces se debe contar el par como una única arista.

Respuesta: 2097


In [30]:
### START CODE HERE
### END CODE HERE

print(n_edges_nodir_A, n_edges_nodir_lib)

### 1.b.4) Cantidad de arcos mutuos

Calcular la cantidad de arcos mutuos en la red (es decir, la cantidad de parejas $(u,v)$ donde $\{(u,v),(v,u)\}\subseteq E$
y $u,v\in V$). Esto significa que si existen ambas $(u,v)\in E$ y $(v,u)\in E$, debe contarse el par como mutuo.

Respuesta: 910

In [31]:
help(g.as_undirected)

In [32]:
### START CODE HERE
### END CODE HERE

print(n_edges_mutual_A, n_edges_mutual_lib)

### 1.b.5) Empleados que no recibieron ningún correo

Calcular la cantidad de vértices con $d_v^{\text{in}}=0$, y listar los nombres de los empleados correspondientes.

Respuesta: 3

In [33]:
### START CODE HERE
### END CODE HERE

print(f"Total: {len(no_email_A)}", no_email_A)
print(f"Total: {len(no_email_lib)}", no_email_lib)

### 1.b.6) Empleados que no enviaron ningún correo

Calcular la cantidad de vértices con $d_v^{\text{out}}=0$, y listar los nombres de los empleados correspondientes. 

Respuesta: 9

In [34]:
### START CODE HERE
### END CODE HERE

print(f"Total: {len(no_sent_A)}", no_sent_A)
print(f"Total: {len(no_sent_lib)}", no_sent_lib)

### 1.b.7) Cantidad de empleados contactados por al menos otros 30 empleados

Calcular la cantidad de empleados que fueron contactados por $30$ o más empleados.

Respuesta: 13

In [35]:
### START CODE HERE
### END CODE HERE

print(f"Total: {len(rec30_A)}", rec30_A)
print(f"Total: {len(rec30_lib)}", rec30_lib)

### 1.b.8) Cantidad de empleados que contactaron a al menos otros 30 empleados

Calcular la cantidad de empleados que contactaron a $30$ o más empleados.

Respuesta: 24

In [36]:
### START CODE HERE
### END CODE HERE

print(f"Total: {len(sent30_A)}", sent30_A)
print(f"Total: {len(sent30_lib)}", sent30_lib)

### 1.b.9) Cantidad de triángulos dirigidos

Calcular la cantidad de triángulos dirigidos en $G$. 
Recordar que $G$ es dirigido y la orientación de los triángulos importa, es decir, $i\to j \to k \to i$ es lo mismo que $k \to i \to j \to k $, pero diferente a $i\to k \to j \to i$.


In [37]:
#con matrices
cant_directed_triangles = np.diag(np.linalg.matrix_power(A_bin, 3)).sum() // 3
print(f"Number of directed triangles {cant_directed_triangles}")

A_sim = A_bin | A_bin.T
cant_undirected_triangles = np.diag(np.linalg.matrix_power(A_sim, 3)).sum() // 2
print(f"Number of undirected triangles {cant_undirected_triangles}")

#con igraph
triangles_lib = len(g.cliques(min=3, max=3)) * 3
print(f"Number of triangles find by igraph {triangles_lib}")

### 1.b.10) ¿Es el grafo conexo?

Solo usando grafos, determinar si el grafo dirigido es debilmente y fuertemente conexo. 
En caso de no serlo obtener el subgrafo con la componente conexa más grande (llamada componente gigante).

In [38]:
### START CODE HERE
### END CODE HERE

print("Is strongly connected?", is_strongly)
print("Is weakly connected?", is_weakly)

In [39]:
h = g.subgraph(g.components(mode="weak")[0])

#usamos el mismo layout_g que el grafo original
ig.plot(h, layout=layout_g, bbox=(0, 0, 200, 200),
        edge_color="gray", edge_width=1, edge_lty=1, edge_arrow_size=.5, 
        vertex_size=3, vertex_label=None)

### 1.b.11) Guardar y cargar grafos

Usando el paquete `igraph` guardar el grafo $G$ en formato PAJEK.
Luego cargar una copia del grafo en $G_\text{copia}$.
¿Porqué los dos grafos no son idénticos?

In [40]:
g.write_pajek("enron_from_igraph.net")

!cat "enron_from_igraph.net" | head

In [41]:
g2 = ig.load("enron_from_igraph.net")

In [42]:
#se perdieron los atributos de vértice
print(ig.summary(g))
print(ig.summary(g2))

In [43]:
#pero la estructura de los grafos es la misma
g.isomorphic(g2)

Usando el paquete igraph guardar el grafo  $G$ en formato GRAPHML. 
Luego cargar una copia del grafo en $G_\text{copia}$.
¿Estos son idénticos?

In [44]:
### START CODE HERE
### END CODE HERE

!cat "enron_from_igraph.graphml" | head

In [45]:
g3 = ig.load("enron_from_igraph.graphml")

#no se perdieron los atributos de vértice
# se agrega un atributo de vértice, 'id', obligatorio 
print(ig.summary(g))
print(ig.summary(g3))

#y la estructura de los grafos es la misma
g.isomorphic(g3)

# 2) NetworkX

La idea de esta segunda parte del notebook es familiarizarse con otra biblioteca para la manipulación de grafos: `NetworkX`.


In [46]:
import networkx as nx
import networkx.algorithms.isomorphism as iso

import scipy.io as sio
import numpy as np

## 2.a) Operaciones Básicas

Comenzamos repitiendo las operaciones báscias pero está vez con la nueva biblioteca.

In [47]:
edgelist = [(1,2), (1, 3), (2,3), (3,4), (2,5), (4, 5), (4,6), (4,7), (5,6), (6,7)]
G = nx.from_edgelist(edgelist)

In [48]:
G.nodes()

In [49]:
G.edges()

In [50]:
nx.draw(G, with_labels = True)

In [51]:
G2 = nx.from_edgelist([(1,2), (1, 3), (2, 3), (3, 2)], create_using=nx.DiGraph)

In [52]:
nx.draw(G2, with_labels=True)

In [53]:
new_labels = {1: "apple", 2: "carrot", 3: "banana"}
nx.relabel_nodes(G2, new_labels)

In [54]:
G2.edges()

In [55]:
for el in G2.adjacency():
  print(el)

In [56]:
nx.adj_matrix(G2).todense()

In [57]:
H = G.subgraph([1, 2, 3, 4, 5])
nx.draw(H, with_labels=True)

In [58]:
G.remove_node(4)
G.remove_edge(1, 2)
G.add_node("apple")
G.add_edge("apple", "banana")

In [59]:
nx.draw(G, with_labels=True)

Por más detalles sobre como trabajar con NetworkX, puede referirse a la documentación [Networkx Docs](https://networkx.org/documentation/stable/tutorial.html).

Como habrá podido observar, la sintaxis de networkX es bastante diferente a la de `IGraph`. Para aquellos acostumbrados a trabajar con `Python` quizas la encuenten más similar a otras bibliotecas.

## 2.b) Dataset de emails en Enron

Esta vez, en lugar de cargar el dataset desde Internet y adecuarlo a `Networkx`, vamos a practicar obtenerlo desde `igraph`. 

Para eso, recordamos que en 1.b.11) escribimos el grafo en el archivo "enron_from_igraph.graphml" usando el formato llamado `GraphML`. Ahora, vamos a leerlo desde `networkx`.

In [60]:
G = nx.read_graphml("enron_from_igraph.graphml")

In [61]:
dict_names = dict((k, v.get("name", k)) for k, v in G.nodes(data=True))
G = nx.relabel_nodes(G, dict_names)

### 2.b.1) Análisis del Dataset

No repetimos todo el análisis, solo resumimos los datos del nuevo grafo.

In [62]:
print(nx.info(G))

In [63]:
nx.draw(G, with_labels=True)

Procedemos a comparar los métodos de la biblioteca con las operaciones usando la matrix de adyacencia.

Ahora nos familiarizarnos con operaciones básicas sobre grafos.

Para cada uno de las siguientes partes, realizar los cálculos usando la matriz de adyacencia $A$ (con álegebra de matrices), y usando el grafo (con métodos de `networkx`).



In [64]:
# recordar que en la sección 1.b.1) calculamos las matrices A y A_bin
display(A.shape)
display(A_bin.shape)

### 2.b.2) Cantidad de aristas (arcos)

Calcular la cantidad de aristas (arcos) en la red.
Es decir, la cantidad de parejas únicas ordenadas $(u,v)\in E$ donde $u,v\in V$.

Respuesta: 3007

In [65]:
n_edges_A = A.astype(bool).sum().sum()

n_edges_lib = G.number_of_edges()

print(n_edges_A, n_edges_lib)

Cantida de aristas no dirigidas

### 2.b.3) Cantidad de aristas no dirigidas



Calcular la cantidad de aristas no dirigidas en la red (es decir, la cantidad de parejas únicas no ordenadas $(u,v)\in E$
donde $u,v\in V$. Esto significa que si $(u,v)\in E$ o $(v,u)\in E$, entonces se debe contar el par como una única arista.

Respuesta: 2097


In [66]:
### START CODE HERE
### END CODE HERE

print(n_edges_nodir_A, n_edges_nodir_lib)

### 2.b.4) Cantidad de arcos mutuos

Calcular la cantidad de arcos mutuos en la red (es decir, la cantidad de parejas $(u,v)$ donde $\{(u,v),(v,u)\}\subseteq E$
y $u,v\in V$). Esto significa que si existen ambas $(u,v)\in E$ y $(v,u)\in E$, debe contarse el par como mutual.

Respuesta: 910

In [67]:
### START CODE HERE
### END CODE HERE

print(n_edges_mutual_A, n_edges_mutual_lib)

### 2.b.5) Empleados que no recibieron ningún correo

Calcular la cantidad de vértices con $d_v^{\text{in}}=0$, y listar los nombres de los empleados correspondientes.

Respuesta: 3

In [68]:
no_email_A = [employees[x] for x in np.where(A.sum(axis=0) == 0)[0]]

### START CODE HERE
### END CODE HERE

print(f"Total: {len(no_email_A)}", no_email_A)
print(f"Total: {len(no_email_lib)}", no_email_lib)

### 2.b.6) Empleados que no enviaron ningún correo

Calcular la cantidad de vértices con $d_v^{\text{out}}=0$, y listar los nombres de los empleados correspondientes. 

Respuesta: 9

In [69]:
no_sent_A = [employees[x] for x in np.where(A.sum(axis=1) == 0)[0]]

### START CODE HERE
### END CODE HERE

print(f"Total: {len(no_sent_A)}", no_sent_A)
print(f"Total: {len(no_sent_lib)}", no_sent_lib)

### 2.b.7) Cantidad de empleados contactados por al menos otros 30 empleados

Calcular la cantidad de empleados que fueron contactados por $30$ o más empleados.

Respuesta: 13

In [70]:
#
rec30_A = [employees[x] for x in np.where(A_bin.sum(axis=0) >= 30)[0]]

rec30_lib = [node["name"] for i, node in G.nodes(data=True) if G.in_degree(i) >= 30]
#

print(f"Total: {len(rec30_A)}", rec30_A)
print(f"Total: {len(rec30_lib)}", rec30_lib)

### 2.b.8) Cantidad de empleados que contactaron a al menos otros 30 empleados

Calcular la cantidad de empleados que contactaron a $30$ o más empleados.

Respuesta: 24

In [71]:
### START CODE HERE
### END CODE HERE

print(f"Total: {len(sent30_A)}", sent30_A)
print(f"Total: {len(sent30_lib)}", sent30_lib)

### 2.b.9) Cantidad de triángulos dirigidos

Calcular la cantidad de triángulos dirigidos en $G$. 
Recordar que $G$ es dirigido y la orientación de los triángulos importa, es decir, $i\to j \to k \to i$ es lo mismo que $k \to i \to j \to k $, pero diferente a $i\to k \to j \to i$.


In [72]:
triangles_lib = None # Not implemented for directed graphs

cant_directed_triangles = np.diag(np.linalg.matrix_power(A_bin, 3)).sum() // 3
print(f"Number of directed triangles {cant_directed_triangles}")

A_sim = A_bin | A_bin.T
cant_undirected_triangles = np.diag(np.linalg.matrix_power(A_sim, 3)).sum() // 2
print(f"Number of undirected triangles {cant_undirected_triangles}")

### 2.b.10) ¿Es el grafo conexo?

Solo usando grafos, determinar si el grafo dirigido es débilmente y fuértemente conexo. 
En caso de no serlo obtener el subgrafo con la componente conexa más grande (llamada componente gigante).

In [73]:
print("Is strongly connected?", nx.is_strongly_connected(G))
print("Is weakly connected?", nx.is_weakly_connected(G))

In [74]:
largest_cc = max(nx.weakly_connected_components(G), key=len)
H = G.subgraph(largest_cc)

In [75]:
nx.draw(H, with_labels=True)

### 2.b.11) Guardar y cargar grafos

Usando el paquete `igraph` guardar el grafo $G$ en formato PAJEK.
Luego cargar una copia del grafo en $G_\text{copia}$.
¿Porqué los dos grafos no son identicos?

In [76]:
nx.write_pajek(G, "enron_from_networkx.net")

!cat "enron_from_networkx.net" | head

In [77]:
G2 = nx.read_pajek("enron_from_networkx.net")

In [78]:
G == G2

In [79]:
display(G.edges(data=True))
display(G2.edges(data=True))

In [80]:
nx.is_isomorphic(G, G2)

Ahora guardar el grafo  $G$ en formato GRAPHML. 
Luego cargar una copia del grafo en $G_\text{copia}$.
¿Estos son idénticos?

In [81]:
### START CODE HERE
### END CODE HERE

!cat "enron_from_networkx.graphml" | head

In [82]:
G3 = nx.read_graphml("enron_from_networkx.graphml")

#no se perdieron los atributos de vértice
# se agrega un atributo de vértice, 'id', obligatorio 
print(nx.info(G))
print(nx.info(G3))

print("identicos: ",G == G3)

#la estructura de los grafos es la misma
print("isomorfos: ", nx.is_isomorphic(G, G3))